In [1]:
import pandas as pd
import json
import random
import matplotlib.pyplot as plt
import gc
import copy
import os

In [3]:
data1 = pd.read_csv("task1.csv")
data2 = pd.read_csv("task2.csv")
data = pd.concat([data1,data2],axis = 0)
data = data[[i for i in data if i not in ['example_id','query_id']]]
data = data[data['query_locale'] == 'us']
data = data.drop_duplicates(["query","product_id"])
data = data[[i for i in data if i != "query_locale"]]
data

,query,product_id,esci_label
0,# 2 pencils not sharpened,B0000AQO0O,exact
1,# 2 pencils not sharpened,B0002LCZV4,exact
2,# 2 pencils not sharpened,B00125Q75Y,exact
3,# 2 pencils not sharpened,B001AZ1D3C,exact
4,# 2 pencils not sharpened,B001B097KC,exact
...,...,...,...
1522302,white adirondack chairs,B09C286CSR,exact
1522303,white adirondack chairs,B09CGNKGKQ,exact
1522304,white adirondack chairs,B09CKGY3FW,substitute
1522305,white adirondack chairs,B09D745NK3,exact


In [ ]:
df_processed = copy.deepcopy(data)
df_processed = df_processed.sample(frac = 1).reset_index(drop = True)
query_item_dict = {}
for row in df_processed.values:
    query_item_dict.setdefault(row[0], [])
    query_item_dict[row[0]].append((row[1],row[2]))

#############
for single_query in query_item_dict:
    kind_item_dict = {}
    for single_item, single_kind in query_item_dict[single_query]:
        kind_item_dict.setdefault(single_kind, [])
        kind_item_dict[single_kind].append(single_item)
    query_item_dict[single_query] = kind_item_dict
query_item_dict = {i:j for i,j in query_item_dict.items() if len(j) ==4}
query_item_dict

{'fast weight loss for women without exercise': {'irrelevant': ['B08CSVJJZR',
   'B07BSR92FQ',
   'B01MTJZNQF',
   'B07XBYX773',
   'B004ZNDFIW'],
  'exact': ['B07GT5GWTJ',
   'B07XZKKDJ8',
   'B097V2PD8K',
   'B07YX4WP2F',
   'B074P8MJBL',
   'B01MUBE6XU',
   'B0849R7N2S',
   'B07N6RDKNT',
   'B084YJQDQV',
   'B01NAXDNE6',
   'B084X4QHJY',
   'B075FVJ31F',
   'B07QK5ZGQ5',
   'B07PFLFQCZ',
   'B07QWPB3NF',
   'B07ZQRPLNV',
   'B08FCY3XXY',
   'B07R8GD47V',
   'B08BJGXHBM',
   'B09HNQK786',
   'B08187N66Y',
   'B07GNZNNFN',
   '1646350243',
   'B07ZS53M6J',
   'B08635PWX3',
   'B01BXLYEQ0',
   'B01GP2Y3BS',
   'B09J8CV9P6',
   'B002KQ6TIE',
   'B089V3GYN2',
   'B00D0VY8BS',
   '1790287650',
   'B07X39RHLK'],
  'substitute': ['B00ISAPPLI'],
  'complement': ['0996435441']},
 '1 quart stainless steel saucepan without lid': {'substitute': ['B00DG08B7U',
   'B07VCVDJ5Y',
   'B08GQV6C9L',
   'B0093AUAGW',
   'B06Y3ZPKRR',
   'B075KQ1WYW',
   'B07ZHL6QY6',
   'B075G4YJBT',
   'B000058AK7',
  

In [24]:
len(query_item_dict)

4652

In [84]:
query_item_dict_copy = copy.deepcopy(query_item_dict)
query_item_dict_copy_org = copy.deepcopy(query_item_dict)
        
all_data_tmp = []

choice_part_dict = {"exact":1,"substitute":1,"complement":2,"irrelevant":1}
query_item_dict_1 = {i:j for i,j in query_item_dict_copy.items() if len(j['complement']) >= 2}
query_item_dict_copy = {i:j for i,j in query_item_dict_copy.items() if len(j['complement']) < 2}
for single_query in query_item_dict_1:
    all_start_list = [0,0,0,0] # exact_start, substitute_start, complement_start, irrelevant_start
    cur_rel_item_dict = query_item_dict_1[single_query]
    
    while True:
        cur_data = []
        flag = 0
        for idx, (rel, part_num) in enumerate(choice_part_dict.items()):
            for _ in range(part_num):
                try:
                    cur_data.append((single_query ,cur_rel_item_dict[rel][all_start_list[idx]], rel))
                except:
                    flag = 1
                all_start_list[idx] += 1
        if flag == 1:
            break
        all_data_tmp.append(cur_data)

    
    
choice_part_dict = {"exact":1,"substitute":1,"complement":1,"irrelevant":2}
query_item_dict_2 = {i:j for i,j in query_item_dict_copy.items() if len(j['irrelevant']) >= 2}
query_item_dict_copy = {i:j for i,j in query_item_dict_copy.items() if len(j['irrelevant']) < 2}
for single_query in query_item_dict_2:
    all_start_list = [0,0,0,0] # exact_start, substitute_start, complement_start, irrelevant_start
    cur_rel_item_dict = query_item_dict_2[single_query]
    
    while True:
        cur_data = []
        flag = 0
        for idx, (rel, part_num) in enumerate(choice_part_dict.items()):
            for _ in range(part_num):
                try:
                    cur_data.append((single_query ,cur_rel_item_dict[rel][all_start_list[idx]], rel))
                except:
                    flag = 1
                all_start_list[idx] += 1
        if flag == 1:
            break
        all_data_tmp.append(cur_data)

choice_part_dict_list = [{"exact":2,"substitute":1,"complement":1,"irrelevant":1},
                   {"exact":1,"substitute":2,"complement":1,"irrelevant":1}]
for single_query in query_item_dict_copy_org:
    all_start_list = [0,0,0,0] # exact_start, substitute_start, complement_start, irrelevant_start
    choice_part_dict = random.choice(choice_part_dict_list)
    cur_rel_item_dict = query_item_dict_copy_org[single_query]
    for i in cur_rel_item_dict:
        random.shuffle(cur_rel_item_dict[i])
    while True:
        cur_data = []
        flag = 0
        for idx, (rel, part_num) in enumerate(choice_part_dict.items()):
            for _ in range(part_num):
                try:
                    cur_data.append((single_query ,cur_rel_item_dict[rel][all_start_list[idx]], rel))
                except:
                    flag = 1
                all_start_list[idx] += 1
        if flag == 1:
            break
        all_data_tmp.append(cur_data)

In [91]:
meta_data = pd.read_csv("product_catalogue-v0.3.csv")
meta_data = meta_data[meta_data['product_locale'] == 'us']
meta_data = meta_data[[i for i in meta_data if i in ['product_id','product_title']]]

In [95]:
meta_data_dict = {i:j for i,j in meta_data.values}

### Map the processed data

In [123]:
all_data_tmp_copy = copy.deepcopy(all_data_tmp)
kind_score_dict = {"exact":1,"substitute":0.1,"complement":0.01,"irrelevant":0}
for i in range(len(all_data_tmp_copy)):
    # shuffle ranking to aviod leakage
    random.shuffle(all_data_tmp_copy[i])
all_data_tmp_copy = [[(j[0],f"{idx + 1}. {meta_data_dict[j[1]]}\n",kind_score_dict[j[2]]) for idx,j in enumerate(i)] for i in all_data_tmp_copy]
# shuffle
random.shuffle(all_data_tmp_copy)
all_data_tmp_copy

[[('dr brown bags',
   "1. Dr. Brown's Silicone Baby Bottle and Pacifier Dishwasher Bag - Blue\n",
   0.01),
  ('dr brown bags',
   "2. Dr. Brown's Tummy Grumbles 6 Piece Reusable Snack Bags\n",
   1),
  ('dr brown bags',
   '3. Momcozy Insulated Baby Bottle Bag, Multi-Function Breastmilk Cooler Bag & Lunch Bag, Fit as Wine Carrier or for Milk Bottles Like Dr. Brown, Comotomo, Philips, Nuk, Lansinoh, etc.\n',
   0.1),
  ('dr brown bags',
   '4. Luxja Double-Layer Breastmilk Cooler Bag (Fits 4 Bottles, Up to 9 Ounce), Breastmilk Cooler for Breastmilk Bottles and Accessories (Bag Only), Gray\n',
   0.1),
  ('dr brown bags',
   "5. Dr. Brown's Options+ Baby Bottles, 8 Ounce, Purple, 3 Count\n",
   0)],
 [('divatrim keto fat utilizing weight loss',
   '1. One Shot Keto Pills, Oneshot Keto Fat 1 Shot Advanced Formula Supplement As Seen on TV, Exogenous Ketones for Rapid Ketosis (60 Capsules)\n',
   1),
  ('divatrim keto fat utilizing weight loss',
   '2. (3 Pack) Official Divatrim Keto Adva

In [124]:
all_prompt_list = []
for row in all_data_tmp_copy:
    prompt_kind = random.choice([0,1])
    query = row[0][0]
    all_candidate_str = "\n".join([i[1] for i in row])
    if prompt_kind == 0:
        prompt = f"You are an intelligent shopping assistant that can rank products based on their relevance to the query. The following numbered list contains 5 products. Please rank the products according to their relevance with the query '{query}'. \nProduct List: \n{all_candidate_str}\nYou should output a permutation of 1 to 5. There should be a comma separating two numbers. Each product and its number should appear only once in the output. Only respond with the ranking results. Do not say any word or explanations.\nOutput: "
    else:
        prompt = f"A user has made a query with keyword '{query}'. Given the following numbered list of 5 products, please rank the products according their relevance with the query. \nProduct List: \n{all_candidate_str}You should output a permutation of 1 to 5. There should be a comma separating two numbers. Each product and its number should appear only once in the output. Only respond with the ranking results. Do not say any word or explanations.\nOutput: "
    all_prompt_list.append([prompt,[i[2] for i in row]])
all_prompt_list

In [129]:
data_final = pd.DataFrame(all_prompt_list)
data_final.columns = ['prompt','output_field']
data_final = data_final.sample(frac = 1)
data_final.to_json(f"query_product_rank_data.json", orient="records", lines=True)
data_final.to_csv(f"query_product_rank_data.csv",index = None)